<a href="https://colab.research.google.com/github/Stephanie9606/Multimodal_Deep_Learning/blob/main/binary_bert_memes_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
!pip3 install bert-for-tf2
!pip3 install sentencepiece

     |████████████████████████████████| 41 kB 143 kB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30535 sha256=4960fce2a769bf9c7aa9bc0cc622ec292acc617707360cdd749963782eec972e
  Stored in directory: /root/.cache/pip/wheels/47/b6/e5/8c76ec779f54bc5c2f1b57d2200bb9c77616da83873e8acb53
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19472 sha256=6e3e63aea1d6da894cde5d2b785613b64002ce2c476038c836b27f9e3a7509e1
  Stored in directory: /root/.cache/pip/wheels/0e/fc/d2/a44fff33af0f233d7def6e7de413006d57c10e10ad736fe8f5
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7911 sha256=2386419742350c84d4442e12f6931b251e9a58491ab887b6ec46a5cdc5a039b0
  Stored in directory: /root/.cache/pip/wheels/e1/11/67/33cc51bbee127cb8fb2ba549cd29109b2f22da43ddf9969716
Successfully built bert-for-tf2 params-flow py-params
     |████████████████████████████████| 1.2 MB 5.3 MB/s 


In [3]:
import tensorflow_hub as hub
from tensorflow.keras.models import Model
from bert.tokenization.bert_tokenization import FullTokenizer
from tensorflow.keras.layers import Dense, Input, Dropout, Flatten, LSTM

In [4]:
# Set up the device for GPU usage
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [5]:
# connect to google drive for files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Pre-processing

In [9]:
memesDF = pd.read_csv('/content/drive/MyDrive/top5_memes_tidy.tsv', sep='\t')

In [10]:
memesDF = memesDF.iloc[:, 2:9]

In [11]:
memesDF['AltText'] = memesDF['AltText'].astype(pd.StringDtype())
memesDF['MemeLabel'] = memesDF['MemeLabel'].astype(pd.StringDtype())
memesDF['CaptionText'] = memesDF['CaptionText'].astype(pd.StringDtype())
memesDF['HashId'] = memesDF['HashId'].astype(pd.StringDtype())
memesDF['ImageURL'] = memesDF['ImageURL'].astype(pd.StringDtype())
memesDF['ImagePath'] = memesDF['ImagePath'].astype(pd.StringDtype())
memesDF['Image'] = memesDF['Image'].astype(pd.StringDtype())

In [12]:
memesDF.dtypes

AltText        string
CaptionText    string
ImageURL       string
HashId         string
MemeLabel      string
ImagePath      string
Image          string
dtype: object

In [13]:
memesDF.nunique()

AltText        5899
CaptionText    5887
ImageURL       5944
HashId         5944
MemeLabel         5
ImagePath      5944
Image          5944
dtype: int64

In [14]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [15]:
# Only get two label for Binary Classification
meme2cDF = memesDF[(memesDF['MemeLabel'] == "Scared Cat") | (memesDF['MemeLabel'] == "Who Killed Hannibal")]

In [16]:
# convert to 1 and 0
meme2cDF['MemeLabel'] = np.where(meme2cDF['MemeLabel'] == "Scared Cat", 1, 0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [17]:
# train test split
from sklearn.model_selection import train_test_split

me_trainDF, me_testDF = train_test_split(meme2cDF, test_size=0.2, random_state=15)

In [18]:
y_train = me_trainDF.MemeLabel
y_test = me_testDF.MemeLabel

In [28]:
me_trainDF.head(3)

,AltText,CaptionText,ImageURL,HashId,MemeLabel,ImagePath,Image
3025,Scared Cat | ME WHEN A GIRL TOUCHES ME | image...,ME WHEN A GIRL TOUCHES ME,http://i.imgflip.com/3hmr2j.jpg,7f8afbac3af4ed48b607e40f035f823b91ee4288807858...,1,/content/memes_image/3hmr2j.jpg,3hmr2j.jpg
363,End of the Thread Week | March 7 - 13 | A Beyo...,March 7 - 13,http://i.imgflip.com/2vl23p.jpg,e41e5221ba4223f38c388266290aaddeb590102c9a0e41...,0,/content/memes_image/2vl23p.jpg,2vl23p.jpg
222,Who Killed Hannibal | UBISOFT BLACKBEARD BALAN...,UBISOFT BLACKBEARD BALANCED,http://i.imgflip.com/3myo0y.jpg,d40f0b6d8625ea11ce839a6330c8337f7a30e283e4a7dd...,0,/content/memes_image/3myo0y.jpg,3myo0y.jpg


# Load Pre-trained Model

In [20]:
!pip install transformers
import transformers

     |████████████████████████████████| 4.0 MB 5.3 MB/s 
     |████████████████████████████████| 6.6 MB 36.3 MB/s 
     |████████████████████████████████| 895 kB 37.3 MB/s 
     |████████████████████████████████| 596 kB 38.6 MB/s 
     |████████████████████████████████| 77 kB 5.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [21]:
from transformers import AutoTokenizer,TFBertModel
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert = TFBertModel.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [22]:
# Tokenize the input (takes some time) 
# here tokenizer using from bert-base-cased
x_train = tokenizer(
    text=me_trainDF.CaptionText.tolist(),
    add_special_tokens=True,
    max_length=128,
    truncation=True,
    padding="max_length", 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True
    #verbose = True
    )
x_test = tokenizer(
    text=me_testDF.CaptionText.tolist(),
    add_special_tokens=True,
    max_length=128,
    truncation=True,
    padding="max_length", 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True
    #verbose = True
    )

In [23]:
input_ids = x_train['input_ids']
attention_mask = x_train['attention_mask']

In [29]:
max_len = 128
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
embeddings = bert(input_ids,attention_mask = input_mask)[0] 
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
#out = Dense(128, activation='relu')(out)
#out = tf.keras.layers.Dropout(0.1)(out)
#out = Dense(32,activation = 'relu')(out)
y = Dense(1,activation = 'sigmoid')(out)
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

In [30]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5), 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

In [31]:
train_history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = y_train,
    validation_data = (
    {'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']}, y_test
    ),
    epochs=10,
    batch_size=32
)

Epoch 1/10
59/59 [==============================] - 118s 2s/step - loss: 0.4910 - accuracy: 0.7593 - val_loss: 0.3990 - val_accuracy: 0.8176
Epoch 2/10
59/59 [==============================] - 94s 2s/step - loss: 0.3016 - accuracy: 0.8646 - val_loss: 0.4000 - val_accuracy: 0.8262
Epoch 3/10
59/59 [==============================] - 94s 2s/step - loss: 0.2236 - accuracy: 0.9060 - val_loss: 0.3264 - val_accuracy: 0.8841
Epoch 4/10
59/59 [==============================] - 94s 2s/step - loss: 0.1501 - accuracy: 0.9447 - val_loss: 0.3273 - val_accuracy: 0.8777
Epoch 5/10
59/59 [==============================] - 94s 2s/step - loss: 0.0733 - accuracy: 0.9823 - val_loss: 0.3467 - val_accuracy: 0.8884
Epoch 6/10
59/59 [==============================] - 94s 2s/step - loss: 0.0406 - accuracy: 0.9898 - val_loss: 0.3649 - val_accuracy: 0.8884
Epoch 7/10
59/59 [==============================] - 94s 2s/step - loss: 0.0630 - accuracy: 0.9796 - val_loss: 0.3510 - val_accuracy: 0.8841
Epoch 8/10
59/59 [=